Import necessary libraries

In [ ]:
import pandas as pd 

In [124]:
df = pd.read_csv("./injury.csv")
df.head()

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,player_id,season_type,season,week,team_id,opponent_id,body_part,status,updated,...,abrv,name_1,id_3,year,id_4,cap_number,cash_spent,season_1,player,team
0,129,11555,REG,1,1,1,5.0,Pectoral,Out,2020-07-30T18:00:07,...,ARI,Cardinals,1,2011,20177,462250,450000,1,11555,1
1,133,12850,REG,1,1,1,5.0,Illness,Probable,2020-07-30T18:00:07,...,ARI,Cardinals,1,2011,4403,74100,74100,1,12850,1
2,132,13406,REG,1,1,1,5.0,Groin,Probable,2020-07-30T18:00:07,...,ARI,Cardinals,1,2011,20101,546023,1059092,1,13406,1
3,134,13417,REG,1,1,1,5.0,Hamstring,Probable,2020-07-30T18:00:07,...,ARI,Cardinals,1,2011,22644,386475,420900,1,13417,1
4,70,11437,REG,1,1,2,6.0,Knee,Out,2020-07-30T18:00:07,...,ATL,Falcons,1,2011,11749,641750,450000,1,11437,2


Clean up dataframe by dropping columns we don't need and reordering remaining columns

In [120]:
df = df.drop(["id", "player", "player_id", "season", "team_id", "opponent_id", "id_1", "id_2", "id_3", "id_4", "season_1", "team"], axis=1)
df = df.rename({"name_1": "team"}, axis=1)
df = df.reindex(columns=["name", "position", "position_group", "number", "year", "season_type", "week", "normalized_week", "team", "abrv", "cap_number", "cash_spent", "body_part", "status", "updated", "declared_inactive"])
df.head()

,name,position,position_group,number,year,season_type,week,normalized_week,team,abrv,cap_number,cash_spent,body_part,status,updated,declared_inactive
0,Jim Dray,TE,hybrid,81,2011,REG,1,6,Cardinals,ARI,462250,450000,Pectoral,Out,2020-07-30T18:00:07,False
1,Korey Lindsey,CB,skill,32,2011,REG,1,6,Cardinals,ARI,74100,74100,Illness,Probable,2020-07-30T18:00:07,False
2,Rob Housler,TE,hybrid,84,2011,REG,1,6,Cardinals,ARI,546023,1059092,Groin,Probable,2020-07-30T18:00:07,False
3,DeMarco Sampson,WR,skill,89,2011,REG,1,6,Cardinals,ARI,386475,420900,Hamstring,Probable,2020-07-30T18:00:07,False
4,Corey Peters,DL,line,91,2011,REG,1,6,Falcons,ATL,641750,450000,Knee,Out,2020-07-30T18:00:07,False


Filter for games that were missed during the regulr season due to injury

In [121]:
games_missed = df[(df["declared_inactive"]==True) & (df["season_type"]=="REG")].drop(["declared_inactive"], axis=1)
games_missed = games_missed.sort_values(by=["name"])
games_missed = games_missed.groupby(["name", "position", "position_group", "year", "team", "cap_number", "cash_spent", "body_part"], as_index=False).size().to_frame("size").reset_index()
games_missed = games_missed.rename({"size": "games_missed"}, axis=1)
games_missed

,name,position,position_group,year,team,cap_number,cash_spent,body_part,games_missed
0,A'Shawn Robinson,DL,line,2018,Lions,1427356,925786,Ankle,1
1,A'Shawn Robinson,DL,line,2018,Lions,1427356,925786,Coach's Decision,1
2,A'Shawn Robinson,DL,line,2018,Lions,1427356,925786,Knee,1
3,A'Shawn Robinson,DL,line,2020,Rams,2000000,6794118,Undisclosed,6
4,A.J. Bouye,CB,skill,2018,Jaguars,15500000,13406250,Toe,1
...,...,...,...,...,...,...,...,...,...
5236,Zak DeOssie,LS,line,2019,Giants,735000,1120000,Knee,4
5237,Zane Gonzalez,K,skill,2020,Cardinals,3259000,3259000,Back,4
5238,Zane Gonzalez,K,skill,2021,Panthers,802778,935000,Quad,2
5239,Zane Lewis,CB,skill,2021,Jets,415000,415000,Knee,17


Calculate the projected money lost against the cap and cash spent due to games missed

In [122]:
def calcMoney(injury):
    div = 16
    if injury["year"] >= 2021: div = 17
    cap_number = injury["cap_number"] // div * injury["games_missed"]
    cash_spent = injury["cash_spent"] // div * injury["games_missed"]
    return cap_number, cash_spent

In [123]:
games_missed[["cap_lost", "cash_lost"]] = games_missed.apply(calcMoney, axis=1, result_type="expand")
games_missed

,name,position,position_group,year,team,cap_number,cash_spent,body_part,games_missed,cap_lost,cash_lost
0,A'Shawn Robinson,DL,line,2018,Lions,1427356,925786,Ankle,1,89209,57861
1,A'Shawn Robinson,DL,line,2018,Lions,1427356,925786,Coach's Decision,1,89209,57861
2,A'Shawn Robinson,DL,line,2018,Lions,1427356,925786,Knee,1,89209,57861
3,A'Shawn Robinson,DL,line,2020,Rams,2000000,6794118,Undisclosed,6,750000,2547792
4,A.J. Bouye,CB,skill,2018,Jaguars,15500000,13406250,Toe,1,968750,837890
...,...,...,...,...,...,...,...,...,...,...,...
5236,Zak DeOssie,LS,line,2019,Giants,735000,1120000,Knee,4,183748,280000
5237,Zane Gonzalez,K,skill,2020,Cardinals,3259000,3259000,Back,4,814748,814748
5238,Zane Gonzalez,K,skill,2021,Panthers,802778,935000,Quad,2,94444,110000
5239,Zane Lewis,CB,skill,2021,Jets,415000,415000,Knee,17,414987,414987
